In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from nfp.preprocessing import MolAPreprocessor, GraphSequence

import keras
import keras.backend as K

from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

from keras.layers import (Input, Embedding, Dense, BatchNormalization,
                                 Concatenate, Multiply, Add)

from keras.models import Model, load_model

from nfp.layers import (MessageLayer, GRUStep, Squeeze, EdgeNetwork,
                               ReduceBondToPro, ReduceBondToAtom, GatherAtomToBond, ReduceAtomToPro)
from nfp.models import GraphModel
from cascade.apply import predict_NMR_C_xyz,predict_NMR_H_xyz

Using TensorFlow backend.
/home/svss/.conda/envs/webapp2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/svss/.conda/envs/webapp2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/svss/.conda/envs/webapp2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/svss/.conda/envs/webapp2/l

In [2]:
import os

In [3]:
modelpath_C = os.path.join('cascade', 'trained_model', 'best_model.hdf5')
modelpath_H = os.path.join('cascade', 'trained_model', 'best_model_H_DFTNN.hdf5')

batch_size = 32
atom_means = pd.Series(np.array([0,0,97.74193,0,0,0,0,0,0,0]).astype(np.float64), name='shift')
NMR_model_C = load_model(modelpath_C, custom_objects={'GraphModel': GraphModel,
                                             'ReduceAtomToPro': ReduceAtomToPro,
                                             'Squeeze': Squeeze,
                                             'GatherAtomToBond': GatherAtomToBond,
                                             'ReduceBondToAtom': ReduceBondToAtom})
NMR_model_H = load_model(modelpath_H, custom_objects={'GraphModel': GraphModel,
                                             'ReduceAtomToPro': ReduceAtomToPro,
                                             'Squeeze': Squeeze,
                                             'GatherAtomToBond': GatherAtomToBond,
                                             'ReduceBondToAtom': ReduceBondToAtom})
# NMR_model_C.summary()
# NMR_model_H.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/svss/.conda/envs/webapp2/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [4]:
#Loading DATA/
import glob
files = glob.glob('xyzfiles/*.xyz')
files

['xyzfiles/mol_1.xyz',
 'xyzfiles/mol_5.xyz',
 'xyzfiles/mol_6.xyz',
 'xyzfiles/mol_2.xyz',
 'xyzfiles/mol_4.xyz',
 'xyzfiles/mol_3.xyz',
 'xyzfiles/mol_7.xyz',
 'xyzfiles/mol_8.xyz',
 'xyzfiles/mol_9.xyz',
 'xyzfiles/mol_10.xyz']

In [5]:
# C predicting NMR
pred_data_C = pd.DataFrame()
for i, file in enumerate(files[1:2]):
    try:
        mols, spreadShift = predict_NMR_C_xyz(file, NMR_model_C)
        pred_data_C = pd.concat([pred_data_C,spreadShift])
    except:
        pass

1it [00:00, 971.80it/s]


In [6]:
pred_data_C

,atom_index,mol_id,predicted
0,1,xyzfiles/mol_5.xyz,129.32
1,2,xyzfiles/mol_5.xyz,129.12
2,3,xyzfiles/mol_5.xyz,124.61
3,5,xyzfiles/mol_5.xyz,124.75


In [7]:
# H predicitions
pred_data_H = pd.DataFrame()
for i, file in enumerate(files[1:2]):
    try:
        mols, spreadShift = predict_NMR_H_xyz(file, NMR_model_H)
        pred_data_H = pd.concat([pred_data_H, spreadShift])
    except:
        pass

1it [00:00, 1062.93it/s]


In [8]:
pred_data_C

,atom_index,mol_id,predicted
0,1,xyzfiles/mol_5.xyz,129.32
1,2,xyzfiles/mol_5.xyz,129.12
2,3,xyzfiles/mol_5.xyz,124.61
3,5,xyzfiles/mol_5.xyz,124.75


In [9]:
pred_data_H

,atom_index,mol_id,predicted
0,6,xyzfiles/mol_5.xyz,6.44
1,7,xyzfiles/mol_5.xyz,6.39
2,8,xyzfiles/mol_5.xyz,6.48
3,9,xyzfiles/mol_5.xyz,6.51
